In [1]:
import os
import importlib
import zipfile as zf
import pandas as pd
import numpy as np
import cvm

importlib.reload(cvm)
pd.options.display.max_colwidth = 15
pd.options.display.max_rows = 4

In [2]:
files_names = sorted(os.listdir('./data/raw/'))
files_names

['dfp_cia_aberta_2010.zip',
 'dfp_cia_aberta_2011.zip',
 'dfp_cia_aberta_2012.zip',
 'dfp_cia_aberta_2013.zip',
 'dfp_cia_aberta_2014.zip',
 'dfp_cia_aberta_2015.zip',
 'dfp_cia_aberta_2016.zip',
 'dfp_cia_aberta_2017.zip',
 'dfp_cia_aberta_2018.zip',
 'dfp_cia_aberta_2019.zip',
 'dfp_cia_aberta_2020.zip',
 'dfp_cia_aberta_2021.zip',
 'itr_cia_aberta_2020.zip',
 'itr_cia_aberta_2021.zip',
 'itr_cia_aberta_2022.zip']

In [5]:
date_columns = ['DT_REFER', 'DT_FIM_EXERC']
kwargs = {
    'sep': ';',
    'encoding': 'iso-8859-1',
    'dtype': str,
    'parse_dates': date_columns}

In [9]:
# full scan = 8m 10s -> 5.6 GB df memory space
# formated colums = 4m 30s -> 3.5 GB df memory space
# zstd -> 16.5 seg
# lz4 -> 15.4 seg
# df_all = pd.DataFrame()
for parent_file_name in files_names:
    print(parent_file_name)
    file_path = f'./data/raw/{parent_file_name}'
    parent_file = zf.ZipFile(file_path)
    # print(parent_file.namelist())
    df_year = pd.DataFrame()
    child_file_names = parent_file.namelist()
    for child_file_name in child_file_names[1:]:
        # print(child_parent_file_name)
        child_file = parent_file.open(child_file_name)
        df_child = pd.read_csv(child_file, **kwargs)
        df_child = cvm.process_child_dataframe(df_child)        
        # df_all = pd.concat([df_all, df_child], ignore_index=True)
        df_year = pd.concat([df_year, df_child], ignore_index=True)
    sort_by = [
        'CD_CVM', 'GRUPO_DFP', 'VERSAO', 'ORDEM_EXERC', 'DT_REFER', 'CD_CONTA']
    df_year.sort_values(by=sort_by, ignore_index=True, inplace=True)
    df_year.to_pickle(
        f'./data/processed/{parent_file_name[:-4]}.pkl',
        compression='zstd')
    # df_year.to_feather(f'./data/processed/{parent_file_name[:-4]}.feather')

dfp_cia_aberta_2010.zip
dfp_cia_aberta_2011.zip
dfp_cia_aberta_2012.zip
dfp_cia_aberta_2013.zip
dfp_cia_aberta_2014.zip
dfp_cia_aberta_2015.zip
dfp_cia_aberta_2016.zip
dfp_cia_aberta_2017.zip
dfp_cia_aberta_2018.zip
dfp_cia_aberta_2019.zip
dfp_cia_aberta_2020.zip
dfp_cia_aberta_2021.zip
itr_cia_aberta_2020.zip
itr_cia_aberta_2021.zip
itr_cia_aberta_2022.zip


In [4]:
files_names = sorted(os.listdir('./data/processed/'))
files_names

['dfp_cia_aberta_2010.pkl',
 'dfp_cia_aberta_2011.pkl',
 'dfp_cia_aberta_2012.pkl',
 'dfp_cia_aberta_2013.pkl',
 'dfp_cia_aberta_2014.pkl',
 'dfp_cia_aberta_2015.pkl',
 'dfp_cia_aberta_2016.pkl',
 'dfp_cia_aberta_2017.pkl',
 'dfp_cia_aberta_2018.pkl',
 'dfp_cia_aberta_2019.pkl',
 'dfp_cia_aberta_2020.pkl',
 'dfp_cia_aberta_2021.pkl',
 'itr_cia_aberta_2020.pkl',
 'itr_cia_aberta_2021.pkl',
 'itr_cia_aberta_2022.pkl']

In [5]:
df = pd.DataFrame()
for file_name in files_names:
    dfy = pd.read_pickle(f'./data/processed/{file_name}', compression='zstd')
    dfy.query("CD_CVM == 9512", inplace=True)
    df = pd.concat([df, dfy], ignore_index=True)
df

,CD_CVM,CNPJ_CIA,DENOM_CIA,GRUPO_DFP,VERSAO,DT_REFER,DT_INI_EXERC,DT_FIM_EXERC,ORDEM_EXERC,CD_CONTA,DS_CONTA,ST_CONTA_FIXA,COLUNA_DF,VL_CONTA
0,9512,33.000.167/...,PETROLEO BR...,DF Consolid...,3,2010-12-31 ...,NaT,2009-12-31 ...,-1,1,Ativo Total,True,NaN,3.504189e+11
1,9512,33.000.167/...,PETROLEO BR...,DF Consolid...,3,2010-12-31 ...,NaT,2009-12-31 ...,-1,1.01,Ativo Circu...,True,NaN,7.437357e+10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37112,9512,33.000.167/...,PETROLEO BR...,DF Individu...,1,2021-09-30 ...,2021-01-01,2021-09-30 ...,0,3.99.02.02,PN,False,NaN,5.760000e+03
37113,9512,33.000.167/...,PETROLEO BR...,DF Individu...,1,2021-09-30 ...,2021-07-01,2021-09-30 ...,0,3.99.02.02,PN,False,NaN,2.390000e+03


In [ ]:
df.info()